In [54]:
import pandas as pd
import numpy as np
import math
import json
import re
import matplotlib.pyplot as plt
from datetime import datetime

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
portfolio = pd.read_json('/content/drive/MyDrive/Udacity/portfolio.json',
                         orient='records', lines=True)
profile = pd.read_json('/content/drive/MyDrive/Udacity/profile.json',
                       orient='records', lines=True)
transcript = pd.read_json('/content/drive/MyDrive/Udacity/transcript.json',
                          orient='records', lines=True)

In [57]:
def check_missing(train):
  """
  check if df have missing values

  Input: dataframe
  Output: column name: number of missing values
  """
  for column in train.columns:
    missing = column, train[column].isnull().sum()
    if missing[1] == 0: continue
    print(missing)

## **Portfolio**

In [58]:
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [59]:
#check missing values
check_missing(portfolio)

In [60]:
portfolio.channels.value_counts()

[web, email, mobile, social]    4
[web, email, mobile]            3
[email, mobile, social]         2
[web, email]                    1
Name: channels, dtype: int64

In [61]:
portfolio.duration.value_counts()

7     4
5     2
10    2
4     1
3     1
Name: duration, dtype: int64

In [62]:
portfolio.offer_type.value_counts()

bogo             4
discount         4
informational    2
Name: offer_type, dtype: int64

In [63]:
#create new features
portfolio.rename(columns={'duration': 'duration_days'}, inplace=True)
portfolio["duration_hours"] = portfolio["duration_days"]*24

portfolio['channels_email'] = portfolio['channels'].apply(
    lambda x: 1 if 'email' in x else 0)
portfolio['channels_mobile'] = portfolio['channels'].apply(
    lambda x: 1 if 'mobile' in x else 0)
portfolio['channels_web'] = portfolio['channels'].apply(
    lambda x: 1 if 'web' in x else 0)
portfolio['channels_social'] = portfolio['channels'].apply(
    lambda x: 1 if 'social' in x else 0)

In [64]:
#ohe on offer_type
df_ot = pd.get_dummies( portfolio['offer_type'],prefix='offer_type')
portfolio = pd.concat([portfolio, df_ot], axis=1)

In [65]:
#delete unnecessary columns
if 'channels' in portfolio: del portfolio['channels']

In [66]:
#save csv
portfolio.to_csv("/content/drive/MyDrive/Udacity/portfolio.csv")

# **Profile**

In [67]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [68]:
#size of the df
profile.shape

(17000, 5)

In [69]:
#check missing values
check_missing(profile)

('gender', 2175)
('income', 2175)


In [70]:
#fill empty cells in gender columns
profile.gender = profile.gender.fillna("ND")

In [71]:
profile.gender.value_counts()

M     8484
F     6129
ND    2175
O      212
Name: gender, dtype: int64

In [72]:
profile.income.value_counts()

73000.0     314
72000.0     297
71000.0     294
57000.0     288
53000.0     282
           ... 
116000.0     46
107000.0     45
112000.0     45
117000.0     32
120000.0     13
Name: income, Length: 91, dtype: int64

In [73]:
#create new column based on 'become_member_on' column
profile['became_member_on_y'] = profile['became_member_on'].astype(str).str[:4]
profile['became_member_on_m'] = profile['became_member_on'].astype(str).str[4:6]
profile['became_member_on_d'] = profile['became_member_on'].astype(str).str[6:]
profile['became_member_on_ym'] = profile['became_member_on_y'] + "_" \
                                            + profile['became_member_on_m']

In [74]:
#split age into groups and create new columns
bins = [0, 25, 35, 45, 55, 65, 110]
labels = ['<26', '26-35', '36-45', '46-55', '56-65', '65<']

profile['age_group'] = pd.cut(profile['age'], bins=bins, labels=labels)

In [75]:
#calculate time to complete offer in days
def days_to_months(value):
    """
    Change days to months
    """
    days = value.days
    months = np.round(days / 30.44,0)
    return months

#convert date type
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'],
                                             format='%Y%m%d')

current_date = datetime.now()
profile['member_since'] = profile['became_member_on'].apply(
    lambda x: (current_date - x))
profile['member_since_months'] = profile['member_since'].apply(days_to_months)
profile.head()

,gender,age,id,became_member_on,income,became_member_on_y,became_member_on_m,became_member_on_d,became_member_on_ym,age_group,member_since,member_since_months
0,ND,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2017,02,12,2017_02,NaN,2444 days 17:27:40.036961,80.0
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017,07,15,2017_07,46-55,2291 days 17:27:40.036961,75.0
2,ND,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,2018,07,12,2018_07,NaN,1929 days 17:27:40.036961,63.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2017,05,09,2017_05,65<,2358 days 17:27:40.036961,77.0
4,ND,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,2017,08,04,2017_08,NaN,2271 days 17:27:40.036961,75.0


In [76]:
#save csv
profile.to_csv("/content/drive/MyDrive/Udacity/profile.csv")

# **Transcript**

In [77]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [78]:
def extract_offer_id(value):
    """
    extracting offer id from value columns

    Input: offer_id column
    Output: offer_id value or none
    """
    if 'offer id' in value:
        return value['offer id']
    elif 'offer_id' in value:
        return value['offer_id']
    else:
        return None

# Apply the function to create a new "offer_id" column
transcript['offer_id'] = transcript['value'].apply(extract_offer_id)

In [79]:
def extract_amount(value):
    if 'amount' in value:
        return value['amount']
    else:
        return None

# Apply the function to create a new "offer_id" column
transcript['amount'] = transcript['value'].apply(extract_amount)

In [80]:
def extract_reward(value):
    if 'reward' in value:
        return value['reward']
    else:
        return None

# Apply the function to create a new "offer_id" column
transcript['reward'] = transcript['value'].apply(extract_reward)

In [81]:
transcript['offer_received'] = transcript['event'].apply(
    lambda x: 1 if x == 'offer received' else 0)
transcript['offer_viewed'] = transcript['event'].apply(
    lambda x: 1 if x == 'offer viewed' else 0)
transcript['offer_completed'] = transcript['event'].apply(
    lambda x: 1 if x == 'offer completed' else 0)

#create columns
transcript['transaction_wo_offer'] = (transcript['amount'].notnull()) \
                                      & (transcript['reward'].isnull())
transcript['transaction_wo_offer'] = transcript['transaction_wo_offer'\
                                                ].astype(int)

#lag column amount
transcript['amount'] = transcript.groupby(['person', 'time'])['amount'].fillna(
    method='ffill')
transcript['amount'] = transcript['amount'].fillna(0)
transcript['reward'] = transcript['reward'].fillna(0)


transcript['amount_wo_offer'] = transcript.apply(
    lambda row: row['transaction_wo_offer'] * row['amount'] if
    row['transaction_wo_offer'] else 0, axis=1)
transcript['amount_w_offer'] =  transcript.apply(
    lambda row: row['offer_completed'] * row['amount'] if
    row['offer_completed'] else 0, axis=1)

In [82]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   person                306534 non-null  object 
 1   event                 306534 non-null  object 
 2   value                 306534 non-null  object 
 3   time                  306534 non-null  int64  
 4   offer_id              167581 non-null  object 
 5   amount                306534 non-null  float64
 6   reward                306534 non-null  float64
 7   offer_received        306534 non-null  int64  
 8   offer_viewed          306534 non-null  int64  
 9   offer_completed       306534 non-null  int64  
 10  transaction_wo_offer  306534 non-null  int64  
 11  amount_wo_offer       306534 non-null  float64
 12  amount_w_offer        306534 non-null  float64
dtypes: float64(4), int64(5), object(4)
memory usage: 30.4+ MB


In [83]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [84]:
transcript.time.value_counts()

408    17030
576    17015
504    16822
336    16302
168    16150
       ...  
318      940
330      938
156      914
162      910
150      894
Name: time, Length: 120, dtype: int64

In [85]:
cols = ['person', 'time']

#fill empty offer id
transcript.sort_values(cols, inplace=True)

#lag column amount
transcript['amount'] = transcript.groupby(cols)['amount'].fillna(method='ffill')
transcript['amount'] = transcript['amount'].fillna(0)
transcript['lagged_time-1'] = transcript.groupby('person')['time'].shift(-1)

#delete event transaction
transcript = transcript[transcript['time'] != transcript['lagged_time-1']]

In [86]:
#delete unnecessary columns
if 'value' in transcript: del transcript['value']
if 'lagged_time-1' in transcript: del transcript['lagged_time-1']

In [87]:
#create cumsum and lag columns
cumulative_cols = ['offer_received', 'offer_viewed', 'offer_completed',
                   'transaction_wo_offer', 'amount_wo_offer', 'amount_w_offer',
                   'reward']

for col in cumulative_cols:
    transcript[f'cumsum_{col}'] = transcript.groupby('person')[col].cumsum()
    transcript[f'cumsum_{col}'] = transcript.groupby('person')[f'cumsum_{col}'
    ].shift()



<ipython-input-87-874e9f5d8aca>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript[f'cumsum_{col}'] = transcript.groupby('person')[col].cumsum()


In [88]:
transcript[transcript.event == "offer completed"].head()

,person,event,time,offer_id,amount,reward,offer_received,offer_viewed,offer_completed,transaction_wo_offer,amount_wo_offer,amount_w_offer,cumsum_offer_received,cumsum_offer_viewed,cumsum_offer_completed,cumsum_transaction_wo_offer,cumsum_amount_wo_offer,cumsum_amount_w_offer,cumsum_reward
168413,0009655768c64bdeb2e877511632db8f,offer completed,414,f19421c1d4aa40978ebb69ca19b0e20d,8.57,5.0,0,0,1,0,0.0,8.57,3.0,2.0,0.0,1.0,22.16,0.00,0.0
228423,0009655768c64bdeb2e877511632db8f,offer completed,528,fafdcd668e3743c1bb461111dcafc2a4,14.11,2.0,0,0,1,0,0.0,14.11,4.0,3.0,1.0,1.0,22.16,8.57,5.0
258884,0009655768c64bdeb2e877511632db8f,offer completed,576,2906b810c7d4411798c6938adc9daaa5,10.27,2.0,0,0,1,0,0.0,10.27,4.0,4.0,2.0,2.0,35.72,22.68,7.0
95422,0011e0d4e6b944f998e987f904e8c1e5,offer completed,252,2298d6c36e964ae4a3e7e9706d1fb8c2,11.93,3.0,0,0,1,0,0.0,11.93,2.0,2.0,0.0,1.0,13.49,0.00,0.0
258981,0011e0d4e6b944f998e987f904e8c1e5,offer completed,576,9b98b8c7a33c4b65b9aebfe6a799e6d9,22.05,5.0,0,0,1,0,0.0,22.05,5.0,5.0,1.0,1.0,13.49,11.93,3.0


In [89]:
def df_based_on_event(event_type, cols, time_diff_cols):
  """
  create separate df for each event type

  Input:
  event_type: precise event type
  cols: define cols to filtering
  time_diff_cols: name of new columns

  Output: new dataframe
  """
  df_boe = transcript[transcript.event == event_type][cols]
  df_boe.sort_values(['person', 'time'], inplace=True)
  df_boe['lag_time'] = df_boe.groupby('person')['time'].shift(1)
  df_boe = df_boe.fillna(0)
  df_boe[time_diff_cols] = df_boe["time"] - df_boe["lag_time"]
  if 'lag_time' in df_boe: del df_boe['lag_time']
  return df_boe

cols_w_id = ['person', 'event', 'time', 'offer_id']
cols_wo_id = ['person', 'event', 'time', ]


transcript_received = df_based_on_event("offer received", cols_w_id,
                                        "time_diff_offer_received")
transcript_transactions = df_based_on_event("transaction", cols_wo_id,
                                            "time_diff_transaction")
transcript_completed = df_based_on_event("offer completed", cols_w_id,
                                         "time_diff_offer_received")

transcript_completed.head()

,person,event,time,offer_id,time_diff_offer_received
168413,0009655768c64bdeb2e877511632db8f,offer completed,414,f19421c1d4aa40978ebb69ca19b0e20d,414.0
228423,0009655768c64bdeb2e877511632db8f,offer completed,528,fafdcd668e3743c1bb461111dcafc2a4,114.0
258884,0009655768c64bdeb2e877511632db8f,offer completed,576,2906b810c7d4411798c6938adc9daaa5,48.0
95422,0011e0d4e6b944f998e987f904e8c1e5,offer completed,252,2298d6c36e964ae4a3e7e9706d1fb8c2,252.0
258981,0011e0d4e6b944f998e987f904e8c1e5,offer completed,576,9b98b8c7a33c4b65b9aebfe6a799e6d9,324.0


In [90]:
transcript["offer_id"] = transcript["offer_id"].fillna(0)

In [91]:
def merge_dfs(first_df,second_df, cols):
  """
  function to merge dfs

  Input:
  first_df: define name of first df
  second_df: define name of second df
  cols: list of columns to merge

  Output: merged df
  """
  transcript = pd.merge(first_df, second_df, on=cols, how='left')
  return transcript

merge_dfs(transcript, transcript_completed, cols_w_id)
transcript = merge_dfs(transcript, transcript_received, cols_w_id)
transcript = merge_dfs(transcript, transcript_transactions, cols_wo_id)
transcript = merge_dfs(transcript, transcript_completed, cols_w_id)
transcript.head()

,person,event,time,offer_id,amount,reward,offer_received,offer_viewed,offer_completed,transaction_wo_offer,...,cumsum_offer_received,cumsum_offer_viewed,cumsum_offer_completed,cumsum_transaction_wo_offer,cumsum_amount_wo_offer,cumsum_amount_w_offer,cumsum_reward,time_diff_offer_received_x,time_diff_transaction,time_diff_offer_received_y
0,0009655768c64bdeb2e877511632db8f,offer received,168,5a8bc65990b245e5a138643cd4eb9837,0.00,0.0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.0,NaN,NaN
1,0009655768c64bdeb2e877511632db8f,offer viewed,192,5a8bc65990b245e5a138643cd4eb9837,0.00,0.0,0,1,0,0,...,1.0,0.0,0.0,0.0,0.00,0.0,0.0,NaN,NaN,NaN
2,0009655768c64bdeb2e877511632db8f,transaction,228,0,22.16,0.0,0,0,0,1,...,1.0,1.0,0.0,0.0,0.00,0.0,0.0,NaN,228.0,NaN
3,0009655768c64bdeb2e877511632db8f,offer received,336,3f207df678b143eea3cee63160fa8bed,0.00,0.0,1,0,0,0,...,1.0,1.0,0.0,1.0,22.16,0.0,0.0,168.0,NaN,NaN
4,0009655768c64bdeb2e877511632db8f,offer viewed,372,3f207df678b143eea3cee63160fa8bed,0.00,0.0,0,1,0,0,...,2.0,1.0,0.0,1.0,22.16,0.0,0.0,NaN,NaN,NaN


In [92]:
#save csv
transcript.to_csv("/content/drive/MyDrive/Udacity/transcript.csv")

# **Join df**

In [93]:
#join 3 dataframes
merged_df = pd.merge(transcript, profile, left_on='person', right_on='id',
                     how='left')
df = pd.merge(merged_df, portfolio, left_on='offer_id', right_on='id',
              how='left')
df.head()

,person,event,time,offer_id,amount,reward_x,offer_received,offer_viewed,offer_completed,transaction_wo_offer,...,offer_type,id_y,duration_hours,channels_email,channels_mobile,channels_web,channels_social,offer_type_bogo,offer_type_discount,offer_type_informational
0,0009655768c64bdeb2e877511632db8f,offer received,168,5a8bc65990b245e5a138643cd4eb9837,0.00,0.0,1,0,0,0,...,informational,5a8bc65990b245e5a138643cd4eb9837,72.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0009655768c64bdeb2e877511632db8f,offer viewed,192,5a8bc65990b245e5a138643cd4eb9837,0.00,0.0,0,1,0,0,...,informational,5a8bc65990b245e5a138643cd4eb9837,72.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0009655768c64bdeb2e877511632db8f,transaction,228,0,22.16,0.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009655768c64bdeb2e877511632db8f,offer received,336,3f207df678b143eea3cee63160fa8bed,0.00,0.0,1,0,0,0,...,informational,3f207df678b143eea3cee63160fa8bed,96.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4,0009655768c64bdeb2e877511632db8f,offer viewed,372,3f207df678b143eea3cee63160fa8bed,0.00,0.0,0,1,0,0,...,informational,3f207df678b143eea3cee63160fa8bed,96.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0


In [94]:
df.to_csv("/content/drive/MyDrive/Udacity/df.csv")